<a href="https://colab.research.google.com/github/Bibhas-Dash/Voice-Enhancement-using-Deep-Learning/blob/main/Simple_audio_denoiser_using_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
cd drive/My Drive

/content/drive/My Drive


In [5]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import math
import librosa
from sklearn import preprocessing as pp
%matplotlib notebook

In [6]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="6"
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.33

In [7]:
tf.compat.v1.set_random_seed(123)

In [8]:
n_file=30

max_length = 60

sr=8000

window_size = 1024

hop_length = 512

In [114]:
def loadfile(path, str_tr, flag = 0):
    list_tr = []
    list_stft = []
    list_stft_abs = []
    list_length = []
    z = ['0', '']
    
    for i in range(1,n_file+1):
        if (i == 0):
            j = 0
        else:
            j = int(math.log10(i))
        s, sr = librosa.load(path + 'sp' + z[j] + str(i) + str_tr + '.wav', sr = None)
        if (flag == 1):
            list_tr.append(s)

        #Calculating STFT
        window = np.hanning(window_size)
        
        stft  = librosa.core.spectrum.stft(s, n_fft = window_size, hop_length = hop_length, window=window)
        
        stft_len = stft.shape[1]     

        #Normalizing the data for training
        if(flag==0):
          stft = 2 * np.abs(stft) / np.sum(window)

        #Appending STFT to list
        if (flag == 1):
            list_stft.append(stft)
        
        #Calculating Absolute of STFT
        stft_abs = np.abs(stft)
        
        #Padding zeros to make length 60
        stft_abs = np.pad(stft_abs, ((0,0),(0, max_length-stft_len)), 'constant')

        #stft_abs =pp.normalize([stft_abs])
        
        #Appending abs to list
        list_stft_abs.append(stft_abs)
        
        #Appending time-length of STFT to list
        list_length.append(stft_len)
        
    return list_tr, list_stft, list_stft_abs, list_length

In [115]:
# Import train input and output data

path_cl='/content/drive/MyDrive/Audio Enh. Dataset/clean_noizeus/wav/'
path_ns='/content/drive/MyDrive/Audio Enh. Dataset/airport_10dB/wav/'

In [116]:
#Loading all training noisy speech signals

trx, X, X_abs, X_len = loadfile(path_ns, '_airport_sn10')

#Loading all training clean speech signals

trs, S, S_abs, S_len = loadfile(path_cl, '')

In [117]:
# Define fully connected network structure
num_hidden_1 = 1000 # hidden layer 1
num_hidden_2 = 1000 # hidden layer 2
num_hidden_3 = 1000 # hidden layer 3
num_hidden_4 = 1000 # hidden layer 4
num_hidden_5 = 1000 # hidden layer 5
num_features = 513 # input features
num_output = 513 # number of outputs

In [118]:
tf.compat.v1.disable_eager_execution()
seq_len = tf.compat.v1.placeholder(tf.int32, None)
x = tf.compat.v1.placeholder(tf.float32, [None, max_length, num_features])
y = tf.compat.v1.placeholder(tf.float32, [None, max_length, num_output])
keep_probability = tf.compat.v1.placeholder("float")  # probability for dropouts

In [119]:
def fully_connected_net(data, keep_probability):
    # Weights and bias initialization
    weights = {
        'w1': tf.Variable(tf.compat.v1.random_normal(shape=[num_features, num_hidden_1],stddev= tf.sqrt(2/(num_features+num_hidden_1)))),
        'w2': tf.Variable(tf.compat.v1.random_normal(shape=[num_hidden_1, num_hidden_2],stddev= tf.sqrt(2/(num_hidden_1+num_hidden_2)))),
        'w3': tf.Variable(tf.compat.v1.random_normal(shape=[num_hidden_2, num_hidden_3],stddev= tf.sqrt(2/(num_hidden_2+num_hidden_3)))),
        'w4': tf.Variable(tf.compat.v1.random_normal(shape=[num_hidden_3, num_hidden_4],stddev= tf.sqrt(2/(num_hidden_3+num_hidden_4)))),
        'w5': tf.Variable(tf.compat.v1.random_normal(shape=[num_hidden_4, num_hidden_5],stddev= tf.sqrt(2/(num_hidden_4+num_hidden_5)))),
        'wout': tf.Variable(tf.compat.v1.random_normal(shape=[num_hidden_5, num_output],stddev = tf.sqrt(2/(num_hidden_5 + num_output))))
    }
    biases = {
        'bout': tf.Variable(tf.compat.v1.random_normal([num_output]))
    }
    # Layer 1 with batch normalization
    z1 = data@weights['w1']
    batch_mean1, batch_var1 = tf.nn.moments(z1, [0])
    z1hat = (z1 - batch_mean1) / tf.sqrt(batch_var1+epsilon)
    # Create two new parameters, scale and beta (shift)
    scale1 = tf.Variable(tf.ones([num_hidden_1]))
    beta1 = tf.Variable(tf.zeros([num_hidden_1]))
    z1_hat = tf.nn.batch_normalization(z1, batch_mean1, batch_var1,beta1, scale1,epsilon)
    l1 = tf.nn.dropout(tf.nn.tanh(z1_hat), keep_probability)
    
    # Layer 2 with batch normalization
    z2 = l1@weights['w2']
    batch_mean2, batch_var2 = tf.nn.moments(z2,[0])
    scale2 = tf.Variable(tf.ones([num_hidden_2]))
    beta2 = tf.Variable(tf.zeros([num_hidden_2]))
    z2_hat = tf.nn.batch_normalization(z2,batch_mean2,batch_var2,beta2,scale2,epsilon)
    l2 = tf.nn.dropout(tf.nn.tanh(z2_hat), keep_probability)

    # Layer 3 with batch normalization
    z3 = l2@weights['w3']
    batch_mean3, batch_var3 = tf.nn.moments(z3,[0])
    scale3 = tf.Variable(tf.ones([num_hidden_3]))
    beta3 = tf.Variable(tf.zeros([num_hidden_3]))
    z3_hat = tf.nn.batch_normalization(z3,batch_mean3,batch_var3,beta3,scale3,epsilon)
    l3 = tf.nn.dropout(tf.nn.tanh(z3_hat), keep_probability)

    # Layer 4 with batch normalization
    z4 = l3@weights['w4']
    batch_mean4, batch_var4 = tf.nn.moments(z4,[0])
    scale4 = tf.Variable(tf.ones([num_hidden_4]))
    beta4 = tf.Variable(tf.zeros([num_hidden_4]))
    z4_hat = tf.nn.batch_normalization(z4,batch_mean4,batch_var4,beta4,scale4,epsilon)
    l4 = tf.nn.dropout(tf.nn.tanh(z4_hat), keep_probability)

    # Layer 5 with batch normalization
    z5 = l4@weights['w5']
    batch_mean5, batch_var5 = tf.nn.moments(z5,[0])
    scale5 = tf.Variable(tf.ones([num_hidden_5]))
    beta5 = tf.Variable(tf.zeros([num_hidden_5]))
    z5_hat = tf.nn.batch_normalization(z5,batch_mean5,batch_var5,beta5,scale5,epsilon)
    l5 = tf.nn.dropout(tf.nn.tanh(z5_hat), keep_probability)
    
    # Output Layer
    output = tf.nn.relu(tf.add(tf.matmul(l5, weights['wout']) , biases['bout']))
    return output

In [130]:
# Small batch size for faster convergence
batch_size=4
# Small epsilon value for the batch normalization
epsilon = 1e-3

In [136]:
# train network

prediction = fully_connected_net(x, keep_probability)
cost = tf.compat.v1.losses.mean_squared_error(y,prediction, weights=1.0)
train_step = tf.compat.v1.train.AdamOptimizer(learning_rate= 0.0001).minimize(cost)
saver = tf.compat.v1.train.Saver()
#feedforward and backpropagation
epochs = 700
sess = tf.compat.v1.Session(config=config)
sess.run(tf.compat.v1.global_variables_initializer())
for epoch in range(epochs):
    epoch_loss = 0
    #start_index = 0
    #X_shuffled, S_shuffled = shuffle(X_abs,S_abs)
    random = np.arange(0, len(X_abs), batch_size)
    np.random.shuffle(random)
    for i in range(len(random)):
        start_index = int(random[i])
        end_index = int(start_index + batch_size)
        if end_index > len(X_abs):
            end_index = len(X_abs)
        batch_x = np.array(X_abs[start_index: end_index]).swapaxes(1,2)
        batch_y = np.array(S_abs[start_index: end_index]).swapaxes(1,2)
        seqlen = np.array(X_len[start_index: end_index])
        start_index = end_index + 1
        _, err = sess.run([train_step, cost], feed_dict={x: batch_x, y: batch_y, seq_len: seqlen, keep_probability: 0.7})
        epoch_loss += err
    for j in range(epoch % 10 == 0):
        print('Epoch ',epoch, ' completed out of ',epochs, 'loss: ', epoch_loss)
print('Epoch ',epoch, ' completed out of ',epochs, 'loss: ', epoch_loss)
saver.save(sess, 'model2/')


Epoch  0  completed out of  700 loss:  7.0365296602249146
Epoch  10  completed out of  700 loss:  6.470064997673035
Epoch  20  completed out of  700 loss:  5.963576912879944
Epoch  30  completed out of  700 loss:  5.4915430545806885
Epoch  40  completed out of  700 loss:  5.0425132513046265
Epoch  50  completed out of  700 loss:  4.649830460548401
Epoch  60  completed out of  700 loss:  4.203336775302887
Epoch  70  completed out of  700 loss:  3.864050030708313
Epoch  80  completed out of  700 loss:  3.418289840221405
Epoch  90  completed out of  700 loss:  2.9960726499557495
Epoch  100  completed out of  700 loss:  2.5854645371437073
Epoch  110  completed out of  700 loss:  2.230322629213333
Epoch  120  completed out of  700 loss:  1.8529794216156006
Epoch  130  completed out of  700 loss:  1.4862291812896729
Epoch  140  completed out of  700 loss:  1.246391475200653
Epoch  150  completed out of  700 loss:  0.9822478741407394
Epoch  160  completed out of  700 loss:  0.8138556778430939

'model2/'

In [132]:
# Import test data

path_ns_te='/content/drive/MyDrive/Audio Enh. Dataset/babble_10dB/wav/'

tex, TEX, TEX_abs, TEX_len = loadfile(path_ns_te, '_babble_sn10', flag=1)


In [133]:
# Getting the predictions for test set

for i in range(len(TEX_abs)):
    #epoch_x = np.array(TEX_abs[0: len(TEX_abs)]).swapaxes(1,2)
    epoch_x = np.zeros((1, TEX_abs[i].shape[1], TEX_abs[i].shape[0]))  
    epoch_x[0,:,:] = TEX_abs[i].T
    
    TEM_pred= sess.run(prediction, feed_dict = {x: epoch_x, seq_len : np.array([TEX_len[i]]), keep_probability: 1.1})
#print(TEM_pred[0][4])

In [134]:
# Reconstruct the output files of the test data
for i in range(len(TEX_abs)):
  out1 = (TEM_pred[0,:TEX_len[i],:].T * TEX[i])
  test1_recons = librosa.istft(out1, win_length= 1024, hop_length=512)
  librosa.output.write_wav('/content/drive/MyDrive/Audio Enh. Dataset/Test_op_data/' + str(i+1) + '.wav', test1_recons, sr)    